# 依赖项 ！！只能有一个参数！！
依赖项用于调用其他 依赖项函数、类 或 任何可调用对象。

其中，依赖项的定义方法跟普通路径操作函数类似，不过没有路由装饰器（@app.get()、@app.post() 等）。

依赖项函数：
```python
async def common_parameters(q: str | None = None, skip: int = 0, limit: int = 100):
    return {"q": q, "skip": skip, "limit": limit}
```

在路径操作函数中:
```python
async def read_items(commons: dict = Depends(common_parameters)): # 会调用 common_parameters 函数，并且返回值给 commons 参数
    return commons
```

声明依赖项用 Depends() 包装即可

`**依赖项函数 可以 依赖 另一个依赖项函数， 即嵌套效果**`

In [ ]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

async def common_parameters(
    q: str | None = None, skip: int = 0, limit: int = 100
):
    return {"q": q, "skip": skip, "limit": limit}


@app.get("/items/")
async def read_items(commons: dict = Depends(common_parameters)):
    return commons


@app.get("/users/")
async def read_users(commons: dict = Depends(common_parameters)):
    return commons

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()
"""
这个url调用到的是read_items函数，但是该函数有个依赖项，因此会先调用依赖项里的函数，再执行read_items函数里的:

    带参数q，访问url ---> read_items ----> 传入q到common_parameters -----> 传入q到common_parameters返回值给commons
        -----> 进入read_items函数里 .-----> 执行 return commons

url = 'http://127.0.0.1:8009/items/?q=张三'  
res = requests.get(url) 
res.text
会得到 '{"q":"张三","skip":0,"limit":100}'
"""

# 类作为依赖项
依赖项并不一定是函数，也可以是类。 `关键是 “可调用”`

类是可调用的， user = User() 当实例化一个类时，实际上就是调用了这个User类里的 __init__() 方法。


In [ ]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

# 创建一个类，该类要有 __init__ 方法，这样才可调用
class CommonQueryParams:
    def __init__(self, q: str | None = None, skip: int = 0, limit: int = 100):
        self.q = q
        self.skip = skip
        self.limit = limit


@app.get("/items/")  # 将类作为依赖项，以此来实例化 并且 赋值给 commons，注意这里commons的类型也是 CommonQueryParams
                     # 但其实也可以不用特意声明类型，对于一个未被初始化的变量，赋予它什么值就会自动成为 什么类型
async def read_items(commons: CommonQueryParams = Depends(CommonQueryParams)):
    return commons


@app.get("/users/")
async def read_users(commons: CommonQueryParams = Depends(CommonQueryParams)):
    return commons

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

# 子依赖项 （类似嵌套依赖）
意思是，依赖项函数里，依赖于另一个依赖项函数


In [ ]:
import uvicorn
from fastapi import Depends, FastAPI
app = FastAPI()

# 依赖项函数
def query_extractor(q: str | None = None):
    return q

# 另一个依赖项函数，且里面的变量q有一个依赖项依赖于上面定义的函数
def query_or_cookie_extractor(
    q: str = Depends(query_extractor),
    last_query: str | None = "last query",
):
    if not q:
        return last_query
    return q

@app.get("/items/")
async def read_query(query_or_default: str = Depends(query_or_cookie_extractor)):
    return {"q_or_query": query_or_default}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='127.0.0.1', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

# 路径装饰器依赖项

有时，我们并不需要在路径操作函数中使用依赖项的返回值。

或者说，有些依赖项不返回值。

但仍要执行或解析该依赖项。

对于这种情况，不必在声明路径操作函数的参数时使用 Depends，而是可以在路径操作装饰器中添加一个由 dependencies 组成的 list。

**无论路径装饰器依赖项是否返回值，路径操作都不会使用这些值**：这是为了避免有些编译器会警告未使用的变量。

In [ ]:
import uvicorn
from fastapi import Depends, FastAPI, Header, HTTPException
app = FastAPI()


async def verify_token(x_token: str = Header()):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: str = Header()):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key

# 在装饰器里添加 dependencies 的list 来存放多个依赖
@app.get("/items/", dependencies=[Depends(verify_token), Depends(verify_key)]) 
async def read_items():
    return [{"item": "Foo"}, {"item": "Bar"}]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

# 全局依赖项
有些依赖项要被用到所有的路径或者路径操作函数中

为了避免一个一个去添加，可以直接在 FastAPI 实例化时，添加一个参数 dependencies ，并传入一个 由 Depends 组成的 list

```python
app = FastAPI(dependencies=[Depends(dependFunction1), Depends(dependFunction2)])
```

In [ ]:
import uvicorn
from fastapi import Depends, FastAPI, Header, HTTPException

async def verify_token(x_token: str = Header()):
    if x_token != "fake-super-secret-token":
        raise HTTPException(status_code=400, detail="X-Token header invalid")


async def verify_key(x_key: str = Header()):
    if x_key != "fake-super-secret-key":
        raise HTTPException(status_code=400, detail="X-Key header invalid")
    return x_key

app = FastAPI(dependencies=[Depends(verify_token), Depends(verify_key)])

@app.get("/items/")
async def read_items():
    return [{"item": "Portal Gun"}, {"item": "Plumbus"}]


@app.get("/users/")
async def read_users():
    return [{"username": "Rick"}, {"username": "Morty"}]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()